In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

plt.rcParams['figure.figsize'] = (16, 10)
plt.rcParams['font.size'] = 11

In [2]:
def parse_csv(filepath):
    with open(filepath, 'r') as f:
        lines = f.readlines()[1:]
    
    data = []
    for line in lines:
        line = line.strip()
        if line and ',' in line and not line.endswith(','):
            parts = line.split(',')
            if len(parts) >= 3:
                try:
                    data.append({'Benchmark': parts[0].strip(), 'Time': float(parts[2])})
                except:
                    continue
    return pd.DataFrame(data)

baseline = parse_csv('BASELINE_bench.csv')
custom = parse_csv('CUSTOM_SIMD_bench.csv')

merged = baseline.merge(custom, on='Benchmark', suffixes=('_baseline', '_custom'))
merged['improvement'] = ((merged['Time_baseline'] - merged['Time_custom']) / merged['Time_baseline']) * 100

FileNotFoundError: [Errno 2] No such file or directory: 'BASELINE_bench.csv'

In [ ]:
sorted_data = merged.sort_values('improvement', ascending=False)
top10 = sorted_data.head(10)
bottom10 = sorted_data.tail(10)
filtered = pd.concat([top10, bottom10])

In [ ]:
heatmap_data = filtered.set_index('Benchmark')[['improvement']]

plt.figure(figsize=(8, 12))
sns.heatmap(heatmap_data, annot=True, fmt='.1f', cmap='RdYlGn', center=0, 
            cbar_kws={'label': 'Performance Improvement (%)'})
plt.title('CUSTOM_SIMD vs BASELINE Performance (Top/Bottom 10)', fontsize=14, fontweight='bold')
plt.ylabel('')
plt.tight_layout()
plt.show()